<h2><center> Homework №1 </center></h2>

<h3>Contents</h3>
        <li><a href="#1">Environment Setup</a></li>
        <li><a href="#2">Task 1</a></li>
        <li><a href="#3">Task 2</a></li>
        <li><a href="#4">Task 3</a></li>

<a name="1"></a>
<h3>Environment Setup</h3>

In [1]:
import os
import sys

os.environ["SPARK_HOME"]="/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [2]:
import pyspark

In [3]:
conf = (
    pyspark.SparkConf()
        .set("spark.executor.memory", "1g")
        .set("spark.executor.core", "2")
        .set("spark.driver.host", "127.0.0.1")  # if you have some troubles with address binding
        .setMaster("local[2]")
)

sc = pyspark.SparkContext(conf=conf)

In [4]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [2]:
# sc.stop()

<a name="2"></a>
<h3>Task 1</h3>

<h5> 1.1 Расcтояние между заведениями и заданной точкой</h5>

In [5]:
from math import cos, asin, sqrt
# import csv


coordinate = (55.751244, 37.618423)


def data_filter(data_csv):
    data_tuple = data_csv.split(",")
    len_data_tuple = len(data_tuple)
    return data_tuple[1], float(data_tuple[len_data_tuple-4]), float(data_tuple[len_data_tuple-5])


def distance(data_fil_rdd):
    lat1 = data_fil_rdd[1]
    lng1 = data_fil_rdd[2]
    lat2 = coordinate[0]
    lng2 = coordinate[1]
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lng2 - lng1) * p)) / 2
    return data_fil_rdd[0], 12742 * asin(sqrt(a))



FILE_PATH_PLACE = "file:///home/ubuntu/datasets/places.csv"
data_rdd = sc.textFile(FILE_PATH_PLACE)

data_fil_rdd = data_rdd.map(data_filter)
data_dist_rdd = data_fil_rdd.map(distance)
data_dist_rdd.take(10)

[('Шоколадница', 4.481040075007144),
 ('МУ-МУ', 4.411446758630132),
 ('КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА', 4.141565403195976),
 ('Дом 12', 2.9767774781758223),
 ('Чито-Ра', 3.0348415747792776),
 ('Бар- буфет «Николай»', 2.828729205783542),
 ('Флорентини', 2.8885782352411793),
 ('Beer Gik', 1.5653137373473913),
 ('Погребок', 1.5653137373473913),
 ('Пробка Гриль', 1.6499887343355246)]

<h5> 1.2 Расстояние между заведениями</h5>

In [6]:
def distance_between_places(data):
    lat1 = data[0][0][1]
    lng1 = data[0][0][2]
    lat2 = data[1][0][1]
    lng2 = data[1][0][2]
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lng2 - lng1) * p)) / 2
    return data[0][0][0], data[1][0][0], 12742 * asin(sqrt(a))


data_dist_betw_rdd = (
    data_fil_rdd.zipWithIndex()
    .cartesian(data_fil_rdd.zipWithIndex())
    .filter(lambda x: x[0][1] < x[1][1])
    .map(distance_between_places)
)
data_dist_betw_rdd.take(10)

[('Шоколадница', 'МУ-МУ', 0.16132427370787492),
 ('Шоколадница', 'КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА', 0.8442372577392393),
 ('Шоколадница', 'Дом 12', 1.580437549471957),
 ('Шоколадница', 'Чито-Ра', 1.5591072632838934),
 ('Шоколадница', 'Бар- буфет «Николай»', 1.6543599984025603),
 ('Шоколадница', 'Флорентини', 1.594263937811669),
 ('Шоколадница', 'Beer Gik', 3.025434509016017),
 ('Шоколадница', 'Погребок', 3.025434509016017),
 ('Шоколадница', 'Пробка Гриль', 2.9369422593505727),
 ('Шоколадница', 'TEMPO DI PASTA', 2.941067369420882)]

<h5> 1.3 Наиболее близкие и наиболее отдаленные заведения </h5>

closest places

In [7]:
data_sorted_T_rdd = data_dist_betw_rdd.sortBy(lambda x: x[2], ascending = True)
data_sorted_T_rdd.take(10)

[('Beer Gik', 'Погребок', 0.0),
 ('TEMPO DI PASTA', 'Хлеб насущный', 0.0),
 ('Beer Gik', 'Kozlovna', 0.0),
 ('Погребок', 'Kozlovna', 0.0),
 ('Beer Gik', 'Па-Паэлья', 0.0),
 ('Погребок', 'Па-Паэлья', 0.0),
 ('Глав Пив Маг', 'Beermood', 0.0),
 ('Блэк милк', 'Дабл Би', 0.0),
 ('Jeffreys coffee', 'Мегобари', 0.0),
 ('Блэк милк', 'Коленки Пчелы', 0.0)]

the most remote places

In [8]:
data_sorted_F_rdd = data_dist_betw_rdd.sortBy(lambda x: x[2], ascending = False)
data_sorted_F_rdd.take(10)

[('МНИТИ', 'Calabash Club', 5.456078051682729),
 ('МНИТИ', 'Залечь на дно', 5.456078051682729),
 ('МНИТИ', 'Политех', 5.456078051682729),
 ('МНИТИ', 'Антикафе Checkpoint', 5.456078051682729),
 ('МНИТИ', 'Шоколадница', 5.402045734207267),
 ('МНИТИ', 'Мареа', 5.397580177699907),
 ('МНИТИ', 'БИБЛИОТЕКА Shisha Lounge', 5.394562231716176),
 ('МНИТИ', 'Му-Му', 5.394562231716176),
 ('МНИТИ', 'Стейк Хаус «Бизон»', 5.393724300516158),
 ('МНИТИ', 'Jimmy Poy', 5.3820992042757405)]

<a name="3"></a>
<h3> Task 2 </h3>

In [9]:
def json_parser_rev(data):
    import json
    data_dict = json.loads(data)
    return data_dict["asin"], data_dict["overall"]


def json_parser_prod(data):
    try:
        data_dict = eval(data)
        return data_dict["asin"], data_dict["title"]
    except:
        return data_dict["asin"], "unknown"


rating_threshold = 3
rating_threshold_br = sc.broadcast(rating_threshold)


FILE_PATH_REVIEWS = "file:///home/ubuntu/Downloads/Electronics_5.json"
FILE_PATH_PRODUCTS = "file:///home/ubuntu/Downloads/meta_Electronics.json"
OUTPUT_DIR_PATH = "file:///home/ubuntu/datasets/output"


data_products_rdd = (
    sc.textFile(FILE_PATH_PRODUCTS)
    .map(json_parser_prod)
)

data_output_rdd = (
    sc.textFile(FILE_PATH_REVIEWS)
    .map(json_parser_rev)
    .aggregateByKey((0, 0), 
                    (lambda x, value: (x[0] + value, x[1] + 1)), 
                    (lambda x, y: (x[0] + y[0], x[1] + y[1])))
    .mapValues(lambda x: x[0]/x[1])                                 # получили средний рейтинг товаров
    .join(data_products_rdd)                                        # добавили название продуктов
    .map(lambda x: (x[0], x[1][1], x[1][0]))
    .filter(lambda x: x[2] < rating_threshold_br.value)             # оставляем товары с рейтингом < 3
    .sortBy(lambda x: x[2], ascending = True)                       # сортируем по возрастанию рейтингов
)

data_output_rdd.take(10)

[('B001BLN91Y',
  'Maxtor OneTouch 4 - 1 TB USB 2.0 Desktop External Hard Drive STM310005OTA3E1-RK',
  1.0),
 ('B003KIQTXG',
  'NEEWER&reg; Photographic Barn Door &amp; Honeycomb Grid &amp; Gel Set for Alienbees Alienbee Flash',
  1.0),
 ('B00111JODG',
  'StarTech HDMISPL1HH 1 feet Standard HDMI Cable - 1x HDMI (M) to 2x HDMI (F) (Discontinued by Manufacturer)',
  1.0),
 ('B000NNFS4C',
  'RCA DRC8335 DVD Recorder &amp; VCR Combo With Built-In Tuner',
  1.0),
 ('B000H13L4Y', 'ATI TV Wonder 200 PCI Video Card w/PVR Capabilities', 1.0),
 ('B000F1ORW6', 'GE 24746 Futura HDTV Ready Antenna', 1.0),
 ('B001T9N0R6',
  'Zeikos 57-in-1 USB 2.0 Flash Memory Card Reader ZE-CR201',
  1.0),
 ('B00000JBIA', 'Agfa ePhoto SMILE 0.2MP Digital Camera', 1.0),
 ('B0013WI5SS', 'Dynex-DX-AP100 Adapter Mini DVI to Mini-DIN', 1.0),
 ('B004K9CL6S',
  'NEW SYLVANIA HD1Z SDSDHCMMC 720P HARD DRIVE POCKET VIDEO DIGITAL CAMERACAMCORDER W4X DIGITAL ZOOM HDMI &amp; 2 LCD PEACOCK BLUE',
  1.0)]

In [10]:
# Remove the output directory
OUTPUT_DIR_PATH_ = OUTPUT_DIR_PATH.replace("file://", "")
!rm -fR $OUTPUT_DIR_PATH_

In [11]:
# Save as text file
data_output_rdd.map(lambda x: "{}, {}, {}".format(x[0], x[1], x[2])).coalesce(1).saveAsTextFile(OUTPUT_DIR_PATH)

<a name="4"></a>
<h3> Task 3 </h3>

<h6>Скалярное произведение</h6>

In [6]:
from math import sqrt


def csv_parser_reting(data_csv):
    try:
        data_list = data_csv.split(",")
        return data_list[0], (data_list[1], float(data_list[2]))
    except:
        return False


FILE_PATH_RETINGS = "file:///home/ubuntu/datasets/ml-latest-small/ratings.csv"


reting_rdd_q = (
    sc.textFile(FILE_PATH_RETINGS)
    .map(csv_parser_reting)
    .filter(lambda x: x != False)
)

scalar_prod_rdd = (
    reting_rdd_q
    .join(reting_rdd_q)                                     #(analogue JOIN)
    .filter(lambda x: x[1][0][0] != x[1][1][0])             #remove repeats type movie_id1 = mox=vie_id1
    .map(lambda x: ((x[1][0][0], x[1][1][0]), x[1][0][1] * x[1][1][1]))
    #.filter(lambda x: x[0] == ("21", "32"))
    .reduceByKey(lambda x, y: x + y)                        #('21 32', 820.5)
)


scalar_prod_rdd.take(5)

[(('190', '1449'), 28.0),
 (('3788', '3176'), 45.0),
 (('4239', '215'), 155.5),
 (('1247', '92259'), 115.75),
 (('7149', '66203'), 20.0)]

<h6>Нормализация</h6>

In [37]:
from math import sqrt


norm_rdd = (
    sc.textFile(FILE_PATH_RETINGS)
    .map(csv_parser_reting)
    .filter(lambda x: x != False) 
    .map(lambda x: (x[1][0], x[1][1]))
    .map(lambda x: (x[0], x[1]**2))
    .reduceByKey(lambda x, y: x + y)
    .map(lambda x: (x[0], sqrt(x[1])))
    .zipWithIndex()
)


norm_rdd_cartes = (
    norm_rdd
    .cartesian(norm_rdd)
    .filter(lambda x: x[0][1] < x[1][1])
    .map(lambda x: ((x[0][0][0], x[1][0][0]), x[0][0][1]*x[1][0][1]))
    #.filter(lambda x: x[0] == ("21", "32"))
)

norm_rdd_cartes.take(5)

[(('1', '50'), 3620.1532253483415),
 (('1', '70'), 1591.272603924293),
 (('1', '110'), 3751.138777091565),
 (('1', '157'), 594.3253528161154),
 (('1', '163'), 1764.2291517827268)]

<h6>Косинусное сходство</h6>

In [29]:
#import org.apache.spark.storage.StorageLevel


cosim_rdd = (
    scalar_prod_rdd
    .join(norm_rdd_cartes)
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
    #.persist(StorageLevel.DISK_ONLY)
)

cosim_rdd.take(5)

[(('593', '450'), 0.10159164782767661),
 (('83349', '102033'), 0.31583994587664016),
 (('1923', '38992'), 0.09042991813254032),
 (('262', '2114'), 0.13157100311072503),
 (('524', '322'), 0.13756794271879122)]

<h6>Второй вариант реализации нормализации</h6>

<h7>Нормализация ver2</h7>

In [8]:
from math import sqrt


norm_rdd_ver2 = (
    sc.textFile(FILE_PATH_RETINGS)
    .map(csv_parser_reting)
    .filter(lambda x: x != False) 
    .map(lambda x: (x[1][0], x[1][1]))
    .map(lambda x: (x[0], x[1]**2))
    .reduceByKey(lambda x, y: x + y)
    .map(lambda x: (x[0], sqrt(x[1])))
)


norm_rdd_ver2.take(5)

[('1', 58.77499468311333),
 ('50', 61.593424973774596),
 ('70', 27.073972741361768),
 ('110', 63.822018144211015),
 ('157', 10.111874208078342)]

<h7>Косинусное сходство</h7>

In [11]:
cosim_rdd_ver2 = (
    scalar_prod_rdd
    .map(lambda x: (x[0][0], (x[0][1], x[1])))
    .join(norm_rdd_ver2)
    .map(lambda x: (x[1][0][0], (x[0], x[1][1], x[1][0][1])))
    .join(norm_rdd_ver2)
    .map(lambda x: ((x[0], x[1][0][0]), x[1][0][2]/(x[1][1] * x[1][0][1])))
    .filter(lambda x: int(x[0][0]) > int(x[0][1]))
    #.filter(lambda x: x[0] == ("4857", "2759") or x[0] == ("2759", "4857"))
)

cosim_rdd_ver2.take(5)

[(('41724', '1617'), 0.08573642792598343),
 (('41724', '1089'), 0.07152918822181996),
 (('41724', '3861'), 0.24494897427831783),
 (('41724', '2707'), 0.15655607277128739),
 (('41724', '440'), 0.09538239546514966)]

<h6>Продолжение task 3</h6>

<h7>Формирование RDD для фильма с id = 589 </h7>

In [12]:
def csv_parser_movies(data_csv):
    try:
        data_list = data_csv.split(",")
        return data_list[0], data_list[1]
    except:
        False


def check_id(data):
    if data[0][0] == "589":
        return data[0][1], (data[0][0], data[1])
    else:
        return data[0][0], (data[0][1], data[1])
    
    
FILE_PATH_MOVIES = "file:///home/ubuntu/datasets/ml-latest-small/movies.csv"


movies_rdd = (
    sc.textFile(FILE_PATH_MOVIES)
    .map(csv_parser_movies)
    .filter(lambda x: x[0] != "movieId" and x != False)
)


movieid_589_rdd = (
    cosim_rdd_ver2
    .filter(lambda x: x[0][0] == "589" or x[0][1] == "589")
    .map(check_id)
    .join(movies_rdd)
    .map(lambda x: (x[1][0][0], x[0], x[1][0][1], x[1][1]))
    .sortBy(lambda x: x[2], ascending = False)
)


movieid_589_rdd.take(10)

[('589', '480', 0.7199832128407866, 'Jurassic Park (1993)'),
 ('589', '1240', 0.6957236907757963, '"Terminator'),
 ('589', '110', 0.6598270010697838, 'Braveheart (1995)'),
 ('589', '592', 0.6456032142113407, 'Batman (1989)'),
 ('589', '457', 0.6375608832771722, '"Fugitive'),
 ('589', '377', 0.6300924636225894, 'Speed (1994)'),
 ('589',
  '1196',
  0.6185297081659638,
  'Star Wars: Episode V - The Empire Strikes Back (1980)'),
 ('589', '380', 0.6111643445364232, 'True Lies (1994)'),
 ('589', '296', 0.6102841093789623, 'Pulp Fiction (1994)'),
 ('589', '356', 0.6008861125749383, 'Forrest Gump (1994)')]

In [12]:
#  sc.stop()